In [129]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [130]:
data_path = 'rct.csv'
df = pd.read_csv(data_path)
df.head()

,grade,label,rct,tsi,stfi,caliper,moisture,basisweight,cull
0,NL050550,4/16/2017 2:36,123.0,4.88,NaN,15.56,0.00,0.0,119.5
1,NL050550,4/16/2017 4:09,127.0,5.15,NaN,15.56,0.00,0.0,119.5
2,NL250740,1/7/2017 15:55,151.0,4.53,NaN,19.24,0.00,0.0,149.5
3,NL110550,2/19/2017 16:20,121.0,4.53,18.1,9.38,6.21,0.0,119.5
4,NL050550,4/16/2017 0:59,123.0,#DIV/0!,NaN,15.67,#DIV/0!,0.0,119.5


In [131]:
# drops weird moisture values
df = df[df.moisture.str.contains('#DIV/0!') == False]
df = df[df.tsi.str.contains('#DIV/0!') == False]
# drops empty and 0 value rows
df = df[np.isfinite(df['stfi'])]


In [132]:
df['tsi'] = df['tsi'].astype(float)
df['moisture'] = df['moisture'].astype(float)
df['cull'] = df['cull'].astype(float)

In [133]:

dummy_fields = ['grade']
for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
    df = pd.concat([df, dummies], axis=1)


fields_to_drop = ['label', 'grade']
data = df.drop(fields_to_drop, axis=1)
data.head()

,rct,tsi,stfi,caliper,moisture,basisweight,cull,grade_NL050350,grade_NL050420,grade_NL050550,grade_NL050740,grade_NL110550,grade_NL110740,grade_NL110850,grade_NL250550,grade_NL250740,grade_NL550740
3,121.0,4.53,18.10,9.38,6.21,0.00,119.5,0,0,0,0,1,0,0,0,0,0
7,122.0,4.06,19.15,10.82,7.06,26.16,119.5,0,0,1,0,0,0,0,0,0,0
8,76.0,4.56,19.81,10.60,6.74,34.89,69.5,1,0,0,0,0,0,0,0,0,0
9,74.0,4.60,19.30,10.64,7.14,35.26,69.5,1,0,0,0,0,0,0,0,0,0
10,77.0,4.63,20.11,10.53,7.04,35.28,69.5,1,0,0,0,0,0,0,0,0,0


In [134]:
quant_features = ['rct','tsi', 'stfi', 'caliper','moisture','basisweight', 'cull']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = (data[each]).mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std
data.head()

,rct,tsi,stfi,caliper,moisture,basisweight,cull,grade_NL050350,grade_NL050420,grade_NL050550,grade_NL050740,grade_NL110550,grade_NL110740,grade_NL110850,grade_NL250550,grade_NL250740,grade_NL550740
3,-0.408414,-0.847915,-2.318256,-2.536936,-0.419331,-5.673515,-0.220737,0,0,0,0,1,0,0,0,0,0
7,-0.358274,-2.567359,-2.099481,-2.001165,1.284747,-3.042524,-0.220737,0,0,1,0,0,0,0,0,0,0
8,-2.664719,-0.738164,-1.961965,-2.083019,0.643212,-2.164521,-2.851408,1,0,0,0,0,0,0,0,0,0
9,-2.764999,-0.591828,-2.068227,-2.068136,1.445131,-2.127309,-2.851408,1,0,0,0,0,0,0,0,0,0
10,-2.614579,-0.482076,-1.899458,-2.109063,1.244651,-2.125298,-2.851408,1,0,0,0,0,0,0,0,0,0


In [135]:
target_fields = ['rct']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = data.drop(target_fields, axis=1), data[target_fields]

train_features, train_targets = features, targets
val_features, val_targets = features, targets

In [136]:
print(train_features.head())

         tsi      stfi   caliper  moisture  basisweight      cull  \
3  -0.847915 -2.318256 -2.536936 -0.419331    -5.673515 -0.220737   
7  -2.567359 -2.099481 -2.001165  1.284747    -3.042524 -0.220737   
8  -0.738164 -1.961965 -2.083019  0.643212    -2.164521 -2.851408   
9  -0.591828 -2.068227 -2.068136  1.445131    -2.127309 -2.851408   
10 -0.482076 -1.899458 -2.109063  1.244651    -2.125298 -2.851408   

    grade_NL050350  grade_NL050420  grade_NL050550  grade_NL050740  \
3                0               0               0               0   
7                0               0               1               0   
8                1               0               0               0   
9                1               0               0               0   
10               1               0               0               0   

    grade_NL110550  grade_NL110740  grade_NL110850  grade_NL250550  \
3                1               0               0               0   
7                0      

In [137]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        self.lr = learning_rate
        
        self.activation_function = lambda x : 1/(1 + np.exp(-x))   # Replace 0 with your sigmoid calculation.
        
                    
    
    def train(self, features, targets):
        ''' Train the network on batch of features and targets. 
        
            Arguments
            ---------
            
            features: 2D array, each row is one data record, each column is a feature
            targets: 1D array of target values
        
        '''
        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        for X, y in zip(features, targets):
            hidden_inputs = np.dot(X,self.weights_input_to_hidden) # signals into hidden layer
            hidden_outputs = self.activation_function(hidden_inputs) # signals from hidden layer
            
            final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) # signals into final output layer
            final_outputs = final_inputs # signals from final output layer'this
            
            #### Implement the backward pass here ####
            ### Backward pass ###

            error = y - final_outputs # Output layer error is the difference between desired target and actual output.

            
            output_error_term = error * 1

            #hidden_error = hidden_outputs * (1 - hidden_outputs)
            #hidden_error_term = np.dot(hidden_error, self.weights_hidden_to_output) * output_error_term
            hidden_error = np.dot(self.weights_hidden_to_output, error)
            hidden_error_term = hidden_error * hidden_outputs * (1- hidden_outputs)
            #print('hidden error term: ' + str(hidden_error_term))
            
            
            # Weight step (input to hidden)
            delta_weights_i_h += hidden_error_term * X[:,None]
            #print('delta in to hidden: ' + str(delta_weights_i_h))
            
            
            # Weight step (hidden to output)
            hidden_outputs = hidden_outputs[:,None]
            delta_weights_h_o += output_error_term * hidden_outputs
            #print('delta hidden to out: ' + str(delta_weights_h_o))
        self.weights_hidden_to_output += self.lr * delta_weights_h_o/n_records # update hidden-to-output weights with gradient descent step
        self.weights_input_to_hidden += self.lr * delta_weights_i_h/n_records # update input-to-hidden weights with gradient descent step
 
    def run(self, features):

        hidden_inputs = np.dot(features,self.weights_input_to_hidden) # signals into hidden layer
        hidden_outputs = self.activation_function(hidden_inputs) # signals from hidden layer
        
        final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) # signals into final output layer
        final_outputs = (final_inputs) # signals from final output layer 
        
        return final_outputs


In [138]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [139]:
import sys

### Set the hyperparameters here ###
iterations = 1500
learning_rate = 0.00008
hidden_nodes = 2000
output_nodes = 1

N_i = train_features.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for ii in range(iterations):
    # Go through a random batch of 128 records from the training data set
    batch = np.random.choice(train_features.index, size=128)
    X, y = train_features.ix[batch].values, train_targets.ix[batch]['rct']
                             
    network.train(X, y)
    
    # Printing out the training progress
    train_loss = MSE(network.run(train_features).T, train_targets['rct'].values)
    val_loss = MSE(network.run(val_features).T, val_targets['rct'].values)
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations)) \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

c:\users\lmoyer\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()


Progress: 99.9% ... Training loss: 0.087 ... Validation loss: 0.087

In [125]:
mean, std = scaled_features['rct']
xs = []
ys = []
value = []
testFeatures = features.reset_index(drop=True)
testTargets = targets.reset_index(drop=True)

for index, row in features.iterrows():
    #print(row)
    prediction = network.run(row).T*std+mean
    xs.append(prediction)


ys = testTargets['rct'].T*std+mean

In [128]:
with open('weight_in1_grades', 'wb') as f:
    pickle.dump(network.weights_input_to_hidden, f)
with open('weight_out1_grades', 'wb') as f:
    pickle.dump(network.weights_hidden_to_output, f)

In [127]:
import json
# save to file:
with open('variables.json', 'w') as f:
    json.dump(scaled_features, f)